# Google Calendar API

[Descripción general de la API de Google Calendar  |  Google for Developers](https://developers.google.com/calendar/api/guides/overview?hl=es-419)

Guía rapida obtener credenciales de autenticación y los eventos de un calendario:

In [ ]:
# Dependencies
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [1]:
import datetime
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]


def main():
  """Shows basic usage of the Google Calendar API.
  Prints the start and name of the next 10 events on the user's calendar.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("calendar", "v3", credentials=creds)

    # Call the Calendar API
    # now = datetime.datetime.utcnow().isoformat() + "Z"  # 'Z' indicates UTC time
    now = datetime.datetime.now(datetime.UTC).isoformat().rsplit("+", 1)[0] + "Z"
    print("Getting the upcoming 10 events")
    events_result = (
        service.events()
        .list(
            calendarId="primary",
            timeMin=now,
            maxResults=10,
            singleEvents=True,
            orderBy="startTime",
        )
        .execute()
    )
    events = events_result.get("items", [])

    if not events:
      print("No upcoming events found.")
      return

    # Prints the start and name of the next 10 events
    for event in events:
      start = event["start"].get("dateTime", event["start"].get("date"))
      print(start, event["summary"])

  except HttpError as error:
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=69722697557-n13nrkrq7t630sthrt70fucdcmn19hsl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51582%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=egQvmqpsqsYXGEHG5FEcVpKlmFLF9I&access_type=offline
Getting the upcoming 10 events
2024-12-10T10:00:00-06:00 Reunion con el equipo


---

## Clase Calendar 

**Para interactuar con la API de Google Calendar, se puede utilizar la clase `Calendar` que tiene los siguientes métodos:**

- `get_credentials()`: Obtiene las credenciales de autenticación.
- `get_service()`: Obtiene el servicio de Google Calendar.
- `get_calendars_info()`: Obtiene la información de todos los calendarios que el usuario puede editar.
- `create_event_body()`: Crea el cuerpo de un evento.
- `create_allday_event_body()`: Crea el cuerpo de un evento de todo el día.
- `create_event()`: Crea un evento (con el cuerpo del evento).
- `get_current_time()`: Obtiene la fecha y hora actual.
- `plus_time()`: Suma un tiempo a una fecha dada.
- `get_events()`: Obtiene los eventos de un calendario.
- `delete_event()`: Elimina un evento de un calendario.
- `move_event()`: Mueve un evento de un calendario a otro.
- `get_event()`: Obtiene un evento de un calendario.
- `update_event()`: Actualiza un evento de un calendario.

In [27]:
from pprint import pprint

import os.path
from typing import Any, List, Dict, Optional, Literal
from datetime import datetime, timedelta

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/calendar"]

UpdateType = Literal['none', 'all', 'externalOnly']
Colors = Literal['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
Disponibility = Literal['opaque', 'transparent']

class Calendar():
    @classmethod
    def get_credentials(cls, secrets_path: str):
        creds = None
        if os.path.exists(secrets_path+"token.json"):
            creds = Credentials.from_authorized_user_file(secrets_path+"token.json", SCOPES)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    secrets_path+"credentials.json", SCOPES
                )
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open(secrets_path+"token.json", "w") as token:
                token.write(creds.to_json())
            return creds
        
    @classmethod
    def get_service(cls, secrets_path: str = "../secrets/"):
        creds = cls.get_credentials(secrets_path)
        return build("calendar", "v3", credentials=creds)
    
    @classmethod
    def get_calendars_info(cls, service: Any) -> List[Dict[str, str]]:
        calendars = service.calendarList().list().execute()
        data = []
        for item in calendars.get('items', []):
            if item['accessRole'] != 'reader':
                data.append({
                    'id': item['id'], 
                    'summary': item['summary'],
                    'timeZone': item['timeZone']})
        return data
    
    @classmethod
    def create_event_body(cls, 
                          summary: str, 
                          start_time: str,
                          end_time: str,
                          timezone: Optional[str] = 'America/Mexico_City',
                          location: Optional[str] = None,
                          description: Optional[str] = None,
                          recurrence: Optional[List[str]] = None,
                          attendees: Optional[List[Dict[str, str]]] = None,
                          reminders: Optional[Dict[str, Any]] = None,
                          conference_data: Optional[Dict[str, dict]] = None,
                          color: Optional[Colors] = None, 
                          transparency: Optional[Disponibility] = 'opaque'
                        ) -> Dict[str, Any]:
        return {
            "summary": summary,
            "location": location,
            "description": description,
            "start": {"dateTime": start_time, "timeZone": timezone},
            "end": {"dateTime": end_time, "timeZone": timezone},
            "recurrence": recurrence,
            "attendees": attendees,
            "reminders": reminders,
            "conferenceData": conference_data, 
            "colorId": color, 
            "transparency": transparency
        }
    
    @classmethod
    def create_allday_event_body(cls, 
                          summary: str, 
                          start_time: str,
                          end_time: str,
                          timezone: Optional[str] = 'America/Mexico_City',
                          location: Optional[str] = None,
                          description: Optional[str] = None,
                          recurrence: Optional[List[str]] = None,
                          attendees: Optional[List[Dict[str, str]]] = None,
                          reminders: Optional[Dict[str, Any]] = None,
                          conference_data: Optional[Dict[str, dict]] = None,
                          color: Optional[str] = None
                        ) -> Dict[str, Any]:
        return {
            "summary": summary,
            "location": location,
            "description": description,
            "start": {"date": start_time, "timeZone": timezone},
            "end": {"date": end_time, "timeZone": timezone},
            "recurrence": recurrence,
            "attendees": attendees,
            "reminders": reminders,
            "conferenceData": conference_data, 
            "colorId": color
        }
    
    @classmethod
    def create_event(cls, service: Any,
                     body: Dict, 
                     calendarId: str = 'primary', 
                     conferenceData: int = 0) -> str:
    
        event = service.events().insert(calendarId=calendarId, 
                                        body=body, 
                                        conferenceDataVersion=conferenceData).execute()
        return event.get('htmlLink')
    
    @classmethod
    def get_current_time(cls) -> str:
        return datetime.now().astimezone().replace(microsecond=0).isoformat()

    @classmethod
    def plus_time(cls, current_datetime: str, minutes: int) -> str:
        date_object = datetime.fromisoformat(current_datetime).strftime('%Y-%m-%d %H:%M:%S')
        new_date = datetime.strptime(date_object, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=minutes)
        return new_date.replace(microsecond=0).isoformat()
    
    @classmethod
    def get_events(cls, service: Any, calendarId: str = 'primary', 
                   maxResults: int = 10, min_time: Optional[str] = None, 
                   max_time: Optional[str] = None, single_events: Optional[bool] = True, 
                   query: Optional[str] = None
                   ) -> List[Dict[str, str]]:
        events_data = []
        events_result = service.events().list(calendarId=calendarId,
                                              timeMin=min_time if min_time else cls.get_current_time(),
                                              timeMax=max_time,
                                              maxResults=maxResults,
                                              singleEvents=single_events,
                                              q=query,
                                              orderBy='startTime').execute()
        events = events_result.get('items', [])
        try: 
            for event in events:
                events_data.append({
                        'id': event['id'], 
                        'start': event["start"].get("dateTime"), 
                        'end': event["end"].get("dateTime"),
                        'summary': event['summary'], 
                        'htmlLink': event.get('htmlLink'), 
                        'creator': event.get('creator').get('email'),
                        'organizer': event.get('organizer').get('email'),
                        })
        except KeyError as error:
            raise KeyError(f"An error occurred: {error}")
        return events_data
    
    @classmethod
    def delete_event(cls, service: Any,  eventId: str, calendarId: str = 'primary',
                     sendUpdates: Optional[UpdateType] = 'none') -> None:
        try:
            service.events().delete(calendarId=calendarId, eventId=eventId, sendUpdates=sendUpdates).execute()
        except HttpError as error:
            print(f"An error occurred: {error}")

    @classmethod
    def move_event(cls, service: Any, eventId: str,
                   destination: str,
                   origin: str = 'primary', 
                   send_updates: Optional[UpdateType] = 'none') -> str:
        
        updated_event = service.events().move(
            calendarId=origin, eventId=eventId,
            destination=destination,
            sendUpdates=send_updates).execute()
        return updated_event['htmlLink']
    
    @classmethod
    def get_event(cls, service: Any, eventId: str, calendarId: str = 'primary') -> Any:
        event = service.events().get(calendarId=calendarId, eventId=eventId).execute()
        return event

    @classmethod
    def update_event(cls, service: Any, event: Dict[str, Any],
                     calendarId: str = 'primary',
                     send_updates: Optional[UpdateType] = 'none') -> str:
        updated_event = service.events().update(calendarId=calendarId,
                                                eventId=event['id'], body=event,
                                                sendUpdates=send_updates).execute()
        return updated_event['htmlLink']

### Ejemplos

In [3]:
service = Calendar.get_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=69722697557-n13nrkrq7t630sthrt70fucdcmn19hsl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49846%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=O9XUFhvioLancULF8dTweKnhnG11SL&access_type=offline


##### Creación de un evento de 40 min

In [92]:
now = Calendar.get_current_time()
now_plus_40_min= Calendar.plus_time(now, 40)

body = Calendar.create_event_body(
    summary="test event",
    start_time=now,
    end_time=now_plus_40_min,
)

Calendar.create_event(service, body)

'https://www.google.com/calendar/event?eid=Y2M4a3BqNnV1aWFxYnJvN3ZuaWkydWRmYTQgam9yZ2VhbmczM0Bt'

##### Creación de un evento de todo el día

In [97]:
from datetime import datetime, timedelta

now = Calendar.get_current_time()
today = datetime.fromisoformat(now) - timedelta(minutes=(datetime.fromisoformat(now).minute))

body = Calendar.create_allday_event_body(
    summary="all day Event",
    start_time=today.strftime('%Y-%m-%d'),
    end_time=(today + timedelta(days=1)).strftime('%Y-%m-%d'),
)

Calendar.create_event(service, body)

'https://www.google.com/calendar/event?eid=ZDB1ZXNwOXZlNzk0cGs3YW9vbWRzaWtoaGsgam9yZ2VhbmczM0Bt'

##### Creación de un evento con recurrencia

In [99]:
now = Calendar.get_current_time()
now_plus_40_min= Calendar.plus_time(now, 40)

body = Calendar.create_event_body(
    summary="test recurrence event",
    start_time=now,
    end_time=now_plus_40_min,
    recurrence=['RRULE:FREQ=DAILY;COUNT=2']
)

Calendar.create_event(service, body)

'https://www.google.com/calendar/event?eid=MmpmaHIyOXRjMTR0MW5mcHJzcGVwcGZlMjhfMjAyNDEwMjJUMDQxMTAwWiBqb3JnZWFuZzMzQG0'

##### Creación de un evento con invitados

In [100]:
now = Calendar.get_current_time()
now_plus_40_min= Calendar.plus_time(now, 40)

body = Calendar.create_event_body(
    summary="test attendees event",
    start_time=now,
    end_time=now_plus_40_min,
    attendees=[{'email': 'juan@example.com'},
               {'email': 'pedro@example.com'}]
)

Calendar.create_event(service, body)

'https://www.google.com/calendar/event?eid=OHBlZG43cWprZ2pwdDcxNnFyZmluZjVtcWsgam9yZ2VhbmczM0Bt'

##### Creación de un evento con recordatorio

In [101]:
now = Calendar.get_current_time()
now_plus_40_min= Calendar.plus_time(now, 40)

body = Calendar.create_event_body(
    summary="test reminders event",
    start_time=now,
    end_time=now_plus_40_min,
    reminders={
        'useDefault': False,
        'overrides': [
            {'method': 'email', 'minutes': 24 * 60},
            {'method': 'popup', 'minutes': 10},
        ],
    }
)

Calendar.create_event(service, body)

'https://www.google.com/calendar/event?eid=Z3I1MjN2bmwwMmwydjdhZXFwZ2gzMWZnYjggam9yZ2VhbmczM0Bt'

##### Creación de un evento con conferencia de Google Meet

In [102]:
from uuid import uuid4


now = Calendar.get_current_time()
now_plus_40_min = Calendar.plus_time(now, 40)

body = Calendar.create_event_body(
    summary="test event with conference",
    start_time=now,
    end_time=now_plus_40_min,
    conference_data={
        "createRequest": {
            "requestId": str(uuid4()),
            "conferenceSolutionKey": {
                "type": "hangoutsMeet"
            }
        }
    }
)

Calendar.create_event(service, body, conferenceData=1)

'https://www.google.com/calendar/event?eid=MTRnOTQwNmxhM3Q4bzZxbWlsYTU2anFoc2cgam9yZ2VhbmczM0Bt'

##### Creación de un evento con diferente color

In [104]:
now = Calendar.get_current_time()
now_plus_40_min= Calendar.plus_time(now, 40)

body = Calendar.create_event_body(
    summary="test event with different color",
    start_time=now,
    end_time=now_plus_40_min,
    color='10'
)

Calendar.create_event(service, body)

'https://www.google.com/calendar/event?eid=djNkZzduMG91OTBtOGJraDQ4bjRtMjBmYmsgam9yZ2VhbmczM0Bt'

##### Creación de un evento con modificación de disponibilidad

In [108]:
now = Calendar.get_current_time()
now_plus_40_min= Calendar.plus_time(now, 40)

body = Calendar.create_event_body(
    summary="test event with disponibility",
    start_time=now,
    end_time=now_plus_40_min,
    transparency='transparent' # Disponible
    # transparency='opaque' # Ocupado
)

Calendar.create_event(service, body)

'https://www.google.com/calendar/event?eid=c3NqYTUzaGhybDVqdjQ2Yzc5bG51cDR2Y3Mgam9yZ2VhbmczM0Bt'

**Creación de eventos con texto:**

In [45]:
created_event = service.events().quickAdd(
    calendarId='primary',
    text='Salida con amigos').execute()

---

##### Actualización de un evento 

Con la utilidad de obtener un evento por su id, se puede actualizar un evento, y se puede enviar un correo a los invitados notificando la actualización.

In [10]:
from pprint import pprint


events = Calendar.get_events(service)
pprint(events)
event = Calendar.get_event(service, eventId=events[0]['id'], calendarId=events[0]['organizer'])
event['summary'] = 'Updated Event with updates'
event.update({'description': 'Se añade una descripcion al evento'})
event.update({'colorId': '3'})
Calendar.update_event(service, event, calendarId=events[0]['organizer'], send_updates='all')

[{'creator': 'jorgeang33@gmail.com',
  'end': '2024-10-22T14:00:00-06:00',
  'htmlLink': 'https://www.google.com/calendar/event?eid=NTVzM3A1djJjc24xN2lvanVjcDhmczZycXMgam9yZ2VhbmczM0Bt',
  'id': '55s3p5v2csn17iojucp8fs6rqs',
  'organizer': 'jorgeang33@gmail.com',
  'start': '2024-10-22T12:00:00-06:00',
  'summary': 'Updated Event with updates'}]


'https://www.google.com/calendar/event?eid=NTVzM3A1djJjc24xN2lvanVjcDhmczZycXMgam9yZ2VhbmczM0Bt'

---

##### Movimiento de un evento de un calendario a otro
Sin notificación a los invitados.

In [25]:
from pprint import pprint


calendars = Calendar.get_calendars_info(service)
pprint(calendars)
events = Calendar.get_events(service)
event = Calendar.get_event(service, 
                           eventId=events[0]['id'],
                           calendarId=events[0]['organizer'])

Calendar.move_event(service, eventId=event['id'], destination=calendars[0]['id'])

[{'id': '9a1c2703ead6fd80455f9e1eb04158669a60d7ab77a8a29ea9ae8d530954b4f7@group.calendar.google.com',
  'summary': 'Familia',
  'timeZone': 'America/Mexico_City'},
 {'id': 'stdevelopersyelling@gmail.com',
  'summary': 'stdevelopersyelling@gmail.com',
  'timeZone': 'America/Mexico_City'},
 {'id': 'jorgeang33@gmail.com',
  'summary': 'jorgeang33@gmail.com',
  'timeZone': 'America/Mexico_City'}]


'https://www.google.com/calendar/event?eid=N2pmN2JodGlxcmlhN2tqMmExYjg5czAwc24gam9yZ2VhbmczM0Bt'

##### Movimiento de un evento de un calendario a otro
Con notificación a los invitados.

In [ ]:
calendars = Calendar.get_calendars_info(service)
events = Calendar.get_events(service)
event = Calendar.get_event(service, 
                           eventId=events[0]['id'],
                           calendarId=events[0]['organizer'])

Calendar.move_event(service, eventId=event['id'], destination=calendars[1]['id'], send_updates='all')

---

##### Eliminación de un evento

In [33]:
from pprint import pprint


calendars = Calendar.get_calendars_info(service)
pprint(calendars)
events = Calendar.get_events(service)
pprint(events)
event = Calendar.get_event(service, 
                           eventId=events[0]['id'],
                           calendarId=events[0]['organizer'])

Calendar.delete_event(service, eventId=event['id'],
                      calendarId=events[0]['organizer'],
                      sendUpdates='all')

[{'id': '9a1c2703ead6fd80455f9e1eb04158669a60d7ab77a8a29ea9ae8d530954b4f7@group.calendar.google.com',
  'summary': 'Familia',
  'timeZone': 'America/Mexico_City'},
 {'id': 'stdevelopersyelling@gmail.com',
  'summary': 'stdevelopersyelling@gmail.com',
  'timeZone': 'America/Mexico_City'},
 {'id': 'jorgeang33@gmail.com',
  'summary': 'jorgeang33@gmail.com',
  'timeZone': 'America/Mexico_City'}]
[{'creator': 'jorgeang33@gmail.com',
  'end': '2024-10-22T15:00:00-06:00',
  'htmlLink': 'https://www.google.com/calendar/event?eid=N2pmN2JodGlxcmlhN2tqMmExYjg5czAwc24gam9yZ2VhbmczM0Bt',
  'id': '7jf7bhtiqria7kj2a1b89s00sn',
  'organizer': 'jorgeang33@gmail.com',
  'start': '2024-10-22T13:00:00-06:00',
  'summary': 'Evento random'}]
